# 4. TF-IDF(Term Frequency-Inverse Document Frequency)
- 주의 : TF-IDF가 DTM보다 항상 성능이 뛰어나진 않습니다.

## 1) TF-IDF(단어 빈도-역 문서 빈도, Term Frequency-Inverse Document Frequency)
> 단어의 빈도와 역 문서 빈도(문서의 빈도에 특정 식을 취함)를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법

- 주로 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업, 문서 내에서 특정 단어의 중요도를 구하는 작업 등에 쓰일 수 있음

### $tf(d,t)$ : 특정 문서 d에서의 특정 단어 t의 등장 횟수

### $df(t)$ : 특정 단어 t가 등장한 문서의 수

### $idf$ :
\begin{align}
idf = log\big(\frac{N}{1+df(t)}\big),  where N=\textrm{문서의 총 개수}
\end{align}

### $TF-IDF$ := $tf \times idf$

###  Example
- 문서1 : 먹고 싶은 사과
- 문서2 : 먹고 싶은 바나나
- 문서3 : 길고 노란 바나나 바나나
- 문서4 : 저는 과일이 좋아요

|단어|IDF(역 문서 빈도)|
|---|---|
|과일이|ln(4/(1+1)) = 0.693147|
|길고|ln(4/(1+1)) = 0.693147|
|노란|ln(4/(1+1)) = 0.693147|
|먹고|ln(4/(2+1)) = 0.287682|
|바나나|ln(4/(2+1)) = 0.287682|
|사과|ln(4/(1+1)) = 0.693147|
|싶은|ln(4/(2+1)) = 0.287682|
|저는|ln(4/(1+1)) = 0.693147|
|좋아요|ln(4/(1+1)) = 0.693147|

### TF-IDF 구하기

|-|과일이|길고|노란|먹고|바나나|사과|싶은|저는|좋아요|
|---|---|---|---|---|---|---|---|---|---|
|문서1|0|0|0|0.287682|0|0.693147|0.287682|0|0|
|문서2|0|0|0|0.287682|0.287682|0|0.287682|0|0|
|문서3|0|0.693147|0.693147|0|0.575364|0|0|0|0|
|문서4|0.693147|0|0|0|0|0|0|0.693147|0.693147|

## 2) 파이썬으로 TF-IDF 직접 구현하기

In [1]:
import pandas as pd # 데이터프레임 사용을 위해
from math import log # IDF 계산을 위해

docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
] 
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

print(vocab)

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']


In [2]:
N = len(docs) # 총 문서의 수

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc  # t in doc은 True 또는 False로 나와서 숫자1, 0으로 인식
    return log(N/(df + 1))

def tfidf(t, d):
    return tf(t,d)* idf(t)

In [3]:
# TF
result = []
for i in range(N): # 각 문서에 대해서 아래 명령을 수행
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]        
        result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [4]:
# IDF
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns = ["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [5]:
# TF-IDF
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]

        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


- 사실 실제 TF-IDF 구현을 제공하고 있는 많은 패키지들은 패키지마다 식이 조금씩 다르긴 하지만, 위에서 배운 기본 식에서 조정된 식을 사용
    - 위의 기본적인 식을 바탕으로 한 구현에도 여전히 문제점이 존재하기 때문

- Example : 만약 전체 문서의 수 n이 4인데, df(t)의 값이 3인 경우
    - $idf$ = 0 이 되어 더이상 가중치 역할을 수행하지 못함..

- 해결책
\begin{align}
idf = log\big(\frac{N}{df(t)+1} \big) + 1
\end{align}
로 log 항에 1을 더해줘서 $IDF$가 최소 1이상의 값을 가지도록 함(사이킷런 방식)

## 3) 사이킷런을 이용한 DTM과 TF-IDF 실습
- 사이킷런은 TF-IDF를 자동 계산해주는 TfidfVectorizer를 제공
- In 사이킷 런,
\begin{align}
idf = \frac{log\big(\frac{N+1}{df(t)+1} \big)+1}{||log\big(\frac{N+1}{df(t)+1} \big)+1||_2}
\end{align}

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray()) # 코퍼스로부터 각 단어의 빈도 수를 기록한다.
print(vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


케라스로도 DTM과 TF-IDF 행렬을 만들 수 있음 $\Longrightarrow$ Ch.11